In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
import spacy
from models.SIM import SIM

%load_ext autoreload
%autoreload 2

In [2]:
math_train = pd.read_json('../Data/dolphin-number_word_std/number_word_std.dev.json')
math_test = pd.read_json('../Data/dolphin-number_word_std/number_word_std.test.json')

In [3]:
model = SIM()
model.fit(math_train)

SIM()

In [5]:
res = model.predict(math_test)
for _,row in res.iterrows():
    print(row['equations'])
    print(row['predicted_equations'])
    print()